In [14]:
import os

# Make sure the "data" folder exists
if not os.path.exists('data'):
    os.makedirs('data')




In [15]:
from astroquery.gaia import Gaia
from astropy.table import Table

# ADQL query to get Gaia DR3 sources near Ruprecht 147
query_gaia = """
SELECT *
FROM gaiadr3.gaia_source
WHERE 1 = CONTAINS(
        POINT('ICRS', ra, dec),
        CIRCLE('ICRS', 289.074, -16.323, 1.0)
      )
  AND phot_g_mean_mag < 14
"""

# Launch the ADQL query asynchronously
job_gaia = Gaia.launch_job_async(query_gaia)

# Get the results as an Astropy Table
rup147_gaia = job_gaia.get_results()
rup147_gaia.write('data/rup147_gaia.csv', format='csv', overwrite=True)



INFO: Query finished. [astroquery.utils.tap.core]


In [23]:
# Extract only the Gaia DR3 source IDs for crossmatching
galah_data = Table.read('data/rup147_gaia.csv')
galah_data['new_source_id'] = galah_data['source_id']
gaia_source_ids = galah_data['source_id'].tolist()

# Convert the source IDs to an Astropy table to use in the query (to not upload too much data)
# with the actual identifier in Gaia DR3 (source_id)
gaia_source_ids_table = Table([gaia_source_ids], names=['source_id'])
gaia_source_ids_table.write('data/gaia_source_ids_table.csv', format='csv', overwrite=True)

In [21]:
gaia_source_ids_table[:5]  # Display the first 5 source IDs

source_id
int64
4088892600474423808
4088893154546972672
4088893188906716032
4087815667506086656
4087815697559746304


In [25]:
query="""
SELECT *
FROM TAP_UPLOAD.t1 AS g
JOIN gaiadr3.tmass_psc_xsc_best_neighbour AS x
    ON g.source_id = x.source_id
JOIN gaiadr3.tmass_psc AS tm
    ON x.tmass_psc_id = tm.tmass_psc_id
"""


In [26]:
job = Gaia.launch_job_async(query=query, upload_resource=gaia_source_ids_table, upload_table_name="t1")

500 Error 500:
null


HTTPError: Error 500:
null

In [27]:
# Try with only the first 10 IDs
gaia_source_ids_table_small = Table([gaia_source_ids[:10]], names=['source_id'])

job = Gaia.launch_job_async(
    query=query,
    upload_resource=gaia_source_ids_table_small,
    upload_table_name="t1"
)


500 Error 500:
null


HTTPError: Error 500:
null